In [1]:
# Predicting Bee Colonies Survival 

In [2]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import sqlite3
from sqlalchemy import create_engine



In [3]:
## updated 
database_path = Path("./save_the_bees.db")

engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()
sql_query = "SELECT * FROM bee_colonies"

df = pd.read_sql(sql_query, conn)



In [4]:
df = df[['state', 'num_colonies', 'max_colonies', 'lost_colonies',
       'percent_lost', 'added_colonies', 'renovated_colonies',
       'percent_renovated', 'quarter', 'year', 'varroa_mites',
       'other_pests_and_parasites', 'diseases', 'pesticides', 'other',
       'unknown']]
df

,state,num_colonies,max_colonies,lost_colonies,percent_lost,added_colonies,renovated_colonies,percent_renovated,quarter,year,varroa_mites,other_pests_and_parasites,diseases,pesticides,other,unknown
0,Alabama,7000,7000,1800,26,2800,250,4,1,2015,10.0,5.4,0.0,2.2,9.1,9.4
1,Arizona,35000,35000,4600,13,3400,2100,6,1,2015,26.9,20.5,0.1,0.0,1.8,3.1
2,Arkansas,13000,14000,1500,11,1200,90,1,1,2015,17.6,11.4,1.5,3.4,1.0,1.0
3,California,1440000,1690000,255000,15,250000,124000,7,1,2015,24.7,7.2,3.0,7.5,6.5,2.8
4,Colorado,3500,12500,1500,12,200,140,1,1,2015,14.6,0.9,1.8,0.6,2.6,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,West Virginia,7500,8000,1100,14,0,220,3,4,2022,33.4,3.8,0.8,0.0,6.4,0.5
1449,Wisconsin,26000,47000,3500,7,140,380,1,4,2022,23.2,21.4,19.4,17.5,9.9,11.7
1450,Wyoming,19500,21000,3200,15,640,0,0,4,2022,22.9,5.9,4.2,0.0,0.0,7.4
1451,Other,30030,30030,480,2,1190,130,0,4,2022,22.4,18.5,0.0,0.0,0.0,0.7


In [5]:
df.describe()

,num_colonies,max_colonies,lost_colonies,percent_lost,added_colonies,renovated_colonies,percent_renovated,quarter,year,varroa_mites,other_pests_and_parasites,diseases,pesticides,other,unknown
count,1.453000e+03,1.453000e+03,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000,1453.000000
mean,1.238883e+05,1.558856e+05,16694.542326,11.219546,15819.036476,13601.734343,7.158981,2.505850,2018.474191,30.186098,10.937509,3.406676,6.185272,6.083345,3.994907
std,4.384485e+05,5.535112e+05,60673.470186,7.375083,63545.779316,57225.881599,9.038191,1.132782,2.322824,18.861293,13.035092,6.472063,8.959392,6.488208,4.939563
min,1.300000e+03,1.300000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000e+03,9.500000e+03,950.000000,6.000000,380.000000,150.000000,1.000000,1.000000,2016.000000,15.600000,1.900000,0.100000,0.400000,1.800000,0.800000
50%,1.850000e+04,2.300000e+04,2200.000000,10.000000,1600.000000,800.000000,4.000000,3.000000,2018.000000,27.200000,7.000000,1.100000,2.600000,4.100000,2.400000
75%,5.800000e+04,7.900000e+04,7000.000000,14.000000,6000.000000,3700.000000,10.000000,4.000000,2021.000000,42.200000,15.100000,4.200000,8.500000,8.200000,5.400000
max,3.181180e+06,5.153870e+06,502350.000000,65.000000,736920.000000,762550.000000,77.000000,4.000000,2022.000000,98.800000,91.900000,87.400000,73.500000,61.400000,46.200000


In [6]:
# Calculate the IQR for the "other_pests_and_parasites" column
Q1 = df["other_pests_and_parasites"].quantile(0.25)
Q3 = df["other_pests_and_parasites"].quantile(0.75)
IQR = Q3 - Q1
# Define the upper and lower bounds to identify outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
# Get the indices of outliers
outliers_indices = (df["other_pests_and_parasites"] < lower_bound) | (df["other_pests_and_parasites"] > upper_bound)
# Remove outliers from the DataFrame
df = df[~outliers_indices]
# Display the DataFrame without outliers
df

,state,num_colonies,max_colonies,lost_colonies,percent_lost,added_colonies,renovated_colonies,percent_renovated,quarter,year,varroa_mites,other_pests_and_parasites,diseases,pesticides,other,unknown
0,Alabama,7000,7000,1800,26,2800,250,4,1,2015,10.0,5.4,0.0,2.2,9.1,9.4
1,Arizona,35000,35000,4600,13,3400,2100,6,1,2015,26.9,20.5,0.1,0.0,1.8,3.1
2,Arkansas,13000,14000,1500,11,1200,90,1,1,2015,17.6,11.4,1.5,3.4,1.0,1.0
3,California,1440000,1690000,255000,15,250000,124000,7,1,2015,24.7,7.2,3.0,7.5,6.5,2.8
4,Colorado,3500,12500,1500,12,200,140,1,1,2015,14.6,0.9,1.8,0.6,2.6,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,West Virginia,7500,8000,1100,14,0,220,3,4,2022,33.4,3.8,0.8,0.0,6.4,0.5
1449,Wisconsin,26000,47000,3500,7,140,380,1,4,2022,23.2,21.4,19.4,17.5,9.9,11.7
1450,Wyoming,19500,21000,3200,15,640,0,0,4,2022,22.9,5.9,4.2,0.0,0.0,7.4
1451,Other,30030,30030,480,2,1190,130,0,4,2022,22.4,18.5,0.0,0.0,0.0,0.7


In [7]:
# Pre-processing
##df = df[df['other_pests_and_parasites'] < 20]
##df = df[df['other_pests_and_parasites'] > 2]

df

,state,num_colonies,max_colonies,lost_colonies,percent_lost,added_colonies,renovated_colonies,percent_renovated,quarter,year,varroa_mites,other_pests_and_parasites,diseases,pesticides,other,unknown
0,Alabama,7000,7000,1800,26,2800,250,4,1,2015,10.0,5.4,0.0,2.2,9.1,9.4
1,Arizona,35000,35000,4600,13,3400,2100,6,1,2015,26.9,20.5,0.1,0.0,1.8,3.1
2,Arkansas,13000,14000,1500,11,1200,90,1,1,2015,17.6,11.4,1.5,3.4,1.0,1.0
3,California,1440000,1690000,255000,15,250000,124000,7,1,2015,24.7,7.2,3.0,7.5,6.5,2.8
4,Colorado,3500,12500,1500,12,200,140,1,1,2015,14.6,0.9,1.8,0.6,2.6,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,West Virginia,7500,8000,1100,14,0,220,3,4,2022,33.4,3.8,0.8,0.0,6.4,0.5
1449,Wisconsin,26000,47000,3500,7,140,380,1,4,2022,23.2,21.4,19.4,17.5,9.9,11.7
1450,Wyoming,19500,21000,3200,15,640,0,0,4,2022,22.9,5.9,4.2,0.0,0.0,7.4
1451,Other,30030,30030,480,2,1190,130,0,4,2022,22.4,18.5,0.0,0.0,0.0,0.7


In [8]:
df = pd.get_dummies(df, columns=['state'], dtype=int)
df

,num_colonies,max_colonies,lost_colonies,percent_lost,added_colonies,renovated_colonies,percent_renovated,quarter,year,varroa_mites,...,state_Tennessee,state_Texas,state_United States,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,7000,7000,1800,26,2800,250,4,1,2015,10.0,...,0,0,0,0,0,0,0,0,0,0
1,35000,35000,4600,13,3400,2100,6,1,2015,26.9,...,0,0,0,0,0,0,0,0,0,0
2,13000,14000,1500,11,1200,90,1,1,2015,17.6,...,0,0,0,0,0,0,0,0,0,0
3,1440000,1690000,255000,15,250000,124000,7,1,2015,24.7,...,0,0,0,0,0,0,0,0,0,0
4,3500,12500,1500,12,200,140,1,1,2015,14.6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,7500,8000,1100,14,0,220,3,4,2022,33.4,...,0,0,0,0,0,0,0,1,0,0
1449,26000,47000,3500,7,140,380,1,4,2022,23.2,...,0,0,0,0,0,0,0,0,1,0
1450,19500,21000,3200,15,640,0,0,4,2022,22.9,...,0,0,0,0,0,0,0,0,0,1
1451,30030,30030,480,2,1190,130,0,4,2022,22.4,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Model Iteration 1

In [10]:
y = df["percent_lost"]
X = df[["varroa_mites", "pesticides", "diseases", "unknown", "quarter", "other_pests_and_parasites", "year"]]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1 
                                                    )
X_train.shape

(1029, 7)

In [12]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

LogisticRegression(max_iter=200, random_state=1)

In [14]:
classifier.fit(X_train, y_train)

C:\Users\antho\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200, random_state=1)

In [15]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.11661807580174927
Testing Data Score: 0.055232558139534885


In [16]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,13,10
1,13,8
2,13,6
3,4,6
4,4,14
5,8,4
6,4,5
7,8,7
8,8,26
9,8,52


In [17]:
accuracy_score(y_test, predictions)

0.055232558139534885

In [18]:
##Binning Percent_Lost

In [19]:
df["percent_lost"].describe()

count    1373.000000
mean       11.248361
std         7.377686
min         0.000000
25%         6.000000
50%        10.000000
75%        14.000000
max        65.000000
Name: percent_lost, dtype: float64

In [20]:
# Creating Bins for Colonies Lost

In [21]:
bins = [-1, 10, 30, 70]
group_labels = [0, 1, 2]
df["target"] = pd.cut(df["percent_lost"], bins, labels=group_labels)
df.dropna() 
df

,num_colonies,max_colonies,lost_colonies,percent_lost,added_colonies,renovated_colonies,percent_renovated,quarter,year,varroa_mites,...,state_Texas,state_United States,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming,target
0,7000,7000,1800,26,2800,250,4,1,2015,10.0,...,0,0,0,0,0,0,0,0,0,1
1,35000,35000,4600,13,3400,2100,6,1,2015,26.9,...,0,0,0,0,0,0,0,0,0,1
2,13000,14000,1500,11,1200,90,1,1,2015,17.6,...,0,0,0,0,0,0,0,0,0,1
3,1440000,1690000,255000,15,250000,124000,7,1,2015,24.7,...,0,0,0,0,0,0,0,0,0,1
4,3500,12500,1500,12,200,140,1,1,2015,14.6,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,7500,8000,1100,14,0,220,3,4,2022,33.4,...,0,0,0,0,0,0,1,0,0,1
1449,26000,47000,3500,7,140,380,1,4,2022,23.2,...,0,0,0,0,0,0,0,1,0,0
1450,19500,21000,3200,15,640,0,0,4,2022,22.9,...,0,0,0,0,0,0,0,0,1,1
1451,30030,30030,480,2,1190,130,0,4,2022,22.4,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df["target"].describe()

count     1373
unique       3
top          0
freq       721
Name: target, dtype: int64

In [23]:
# Model Iteration 2

In [24]:
y = df["target"]
X = df[["varroa_mites", "pesticides", "diseases", "unknown", "quarter", "other_pests_and_parasites", "year"]]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1 
                                                    )
X_train.shape

(1029, 7)

In [26]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [27]:
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

LogisticRegression(max_iter=200, random_state=1)

In [28]:
classifier.fit(X_train, y_train)

C:\Users\antho\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200, random_state=1)

In [29]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.6637512147716229
Testing Data Score: 0.6686046511627907


In [30]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,1,0
1,0,0
2,0,0
3,0,0
4,0,1
5,0,0
6,0,0
7,0,0
8,0,1
9,0,2


In [31]:
accuracy_score(y_test, predictions)

0.6686046511627907

In [32]:
# Model Iteration 3

In [33]:
y = df["target"]
X = df.drop(columns=["target", "percent_lost", "state_United States", "state_Other", 'num_colonies', 'max_colonies', 'lost_colonies',      'percent_lost', 'added_colonies', 'renovated_colonies', 'percent_renovated', 'quarter', 'year'])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1 
                                                    )
X_train.shape

(1029, 51)

In [35]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [36]:
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

LogisticRegression(max_iter=200, random_state=1)

In [37]:
classifier.fit(X_train, y_train)

C:\Users\antho\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200, random_state=1)

In [38]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.6977648202137998
Testing Data Score: 0.6715116279069767


In [39]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,1,0
1,0,0
2,0,0
3,0,0
4,0,1
5,0,0
6,0,0
7,0,0
8,1,1
9,1,2


In [40]:
accuracy_score(y_test, predictions)

0.6715116279069767

In [41]:
# Model Iteration 4